In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torchaudio
torchaudio.set_audio_backend('sox_io')

import numpy as np
import pandas as pd

import os

In [2]:

from nna import dataimport
from nna import fileUtils
from nna.exp import runutils

import modelarchs  # type: ignore


In [3]:


def repeat_data(data, expected_len) -> np.ndarray:
    '''pad by zeros if it is not divisible expected_len seconds.
    '''
    sr = 48000
    left_over = (data.shape[0]) % (expected_len * sr)

    if left_over != 0:
        missing_element_count = (expected_len * sr) - left_over
        padded_data = np.pad(data[-left_over:], (0, missing_element_count),
                             'constant',
                             constant_values=(0, 0))
        return np.concatenate([data[:-left_over], padded_data])  # type: ignore
    else:
        return data



In [43]:
class pathMap():
    def __init__(self) -> None:
        scratch = '/scratch/enis/data/nna/'
        home = '/home/enis/projects/nna/'
        self.exp_dir = '/home/enis/projects/nna/src/nna/exp/megan/run-5/'
        self.clipping_results_path = (scratch +
                                         'clipping_info/all-merged_2021-02-10/')
        self.output_dir = scratch + 'real/'
        self.file_properties_df_path = scratch + 'database/allFields_dataV4.pkl'
        # model_path= ('/home/enis/projects/nna/src/nna/exp/megan/run-3/'+
        # 'checkpoints_keep/glorious-sweep-57/best_model_56_ROC_AUC=0.8690.pt')
        checkpoints_dir = scratch + 'runs_models/megan/run-5/checkpoints/'
        self.model_path = (
            checkpoints_dir +
            'comic-sweep-29/best_model_20_min_ROC_AUC=0.7202.pt')



def setup_inputs(args):
    index, count = int(args.index), int(args.count)

    region_location = [['dalton', '01'], ['dalton', '02'], ['dalton', '03'],
                       ['dalton', '04'], ['dalton', '05'], ['dalton', '06'],
                       ['dalton', '07'], ['dalton', '08'], ['dalton', '09'],
                       ['dalton', '10'], ['dempster', '11'], ['dempster', '12'],
                       ['dempster', '13'], ['dempster',
                                            '14'], ['dempster', '16'],
                       ['dempster', '17'], ['dempster',
                                            '19'], ['dempster', '20'],
                       ['dempster', '21'], ['dempster',
                                            '22'], ['dempster', '23'],
                       ['dempster', '24'], ['dempster', '25'],
                       ['ivvavik', 'AR01'], ['ivvavik', 'AR02'],
                       ['ivvavik', 'AR03'], ['ivvavik', 'AR04'],
                       ['ivvavik', 'AR05'], ['ivvavik', 'AR06'],
                       ['ivvavik', 'AR07'], ['ivvavik', 'AR08'],
                       ['ivvavik', 'AR09'], ['ivvavik', 'AR10'],
                       ['ivvavik', 'SINP01'], ['ivvavik', 'SINP02'],
                       ['ivvavik', 'SINP03'], ['ivvavik', 'SINP04'],
                       ['ivvavik', 'SINP05'], ['ivvavik', 'SINP06'],
                       ['ivvavik', 'SINP07'], ['ivvavik', 'SINP08'],
                       ['ivvavik', 'SINP09'], ['ivvavik', 'SINP10']]

    return region_location[index:index + count]


def setup(args):

    pathmap = pathMap()

    os.chdir(pathmap.exp_dir)

    file_properties_df = pd.read_pickle(pathmap.file_properties_df_path)

    device = 'cuda:' + str(args.gpu)
    device = torch.device(device)

    CATEGORY_COUNT = 9
    # '1.1.10','1.1.7'
    maxMelLen = 938
    ToTensor_ins = modelarchs.ToTensor(maxMelLen, 48000, device)
    transformCompose = transforms.Compose([
        ToTensor_ins,
    ])

    h_w = [128, 938]

    config = {}
    config['label_names'] = ['1-0-0',
                            '1-1-0',
                            '1-1-10',
                            '1-1-7',
                            '0-0-0',
                            '1-3-0',
                            '1-1-8',
                            '0-2-0',
                            '3-0-0',
                            ]
    config['v_str'] = 'multi9-V1'
    config['fc_1_size'] = 64
    config['CNN_kernel_size'] = 12
    config['CNN_filters_1'] = 6
    config['expected_len'] = 10
    config['device'] = device

    output_shape = (CATEGORY_COUNT,)

    model_saved = modelarchs.singleconv1dModel(
        out_channels=config['CNN_filters_1'],
        h_w=(1, h_w[0] * h_w[1]),
        fc_1_size=config['fc_1_size'],
        kernel_size=config['CNN_kernel_size'],
        output_shape=output_shape)

    model_saved.load_state_dict(
        torch.load(pathmap.model_path, map_location=config['device']))
    model_saved.eval().to(config['device'])

    return model_saved, transformCompose, config, file_properties_df, pathmap






In [80]:
def prepare_dataloader_from_audio_ins(audio_ins, config, transformCompose):
    audio_ins.load_data()
    audio_ins.pick_channel_by_clipping(config['expected_len'])
    input_file_data = repeat_data(audio_ins.data, config['expected_len'])

    # divide to 10 second excerpts
    input_file_data = input_file_data.reshape(-1, 480000)
    input_file_data = torch.from_numpy(input_file_data).float()
    dataset = {
        'predict':
            runutils.audioDataset(input_file_data,
                                  None,
                                  transform=transformCompose)
    }
    dataloader = {
        'predict':
            torch.utils.data.DataLoader(dataset['predict'],
                                        shuffle=False,
                                        batch_size=1)
    }
    return dataloader

def prepare_dataloader_from_array(input_data):

    # divide to 10 second excerpts
    input_file_data = input_data.reshape(-1, 480000)
    input_file_data = torch.from_numpy(input_file_data).float()
    dataset = {
        'predict':
            runutils.audioDataset(input_file_data,
                                  None,
                                  transform=transformCompose)
    }
    dataloader = {
        'predict':
            torch.utils.data.DataLoader(dataset['predict'],
                                        shuffle=False,
                                        batch_size=1)
    }
    return dataloader



def single_file_inference(dataloader, config, model_saved):
    outputs = []
    for inputs, labels in dataloader['predict']:
        del labels
        inputs = inputs.float().to(config['device'])
        output = model_saved(inputs)
        output = output.to('cpu')
        index = output.data.numpy()
        outputs.append(index)
    outputs = np.concatenate(outputs)
    return outputs


def save_results_disk(outputs, audio_ins, label_names, v_str,
                      file_properties_df, pathmap):
    # label_names = ['1-1-10', '1-1-7']
    # v_str = 'V3'
    file_names = output_file_names(audio_ins, label_names, v_str,
                                   file_properties_df, pathmap)
    for i, file_name in enumerate(file_names):
        file_name.parent.mkdir(parents=True, exist_ok=True)
        file_name = file_name.with_suffix('.npy')
        np.save(str(file_name), outputs[:, i])
    audio_ins.data = None


def output_file_names(audio_ins, label_names, v_str, file_properties_df,
                      pathmap):
    row = file_properties_df.loc[audio_ins.path]
    file_names = []
    for _, label_name in enumerate(label_names):
        sub_directory_addon = v_str + '-' + label_name
        file_name_addon = sub_directory_addon
        file_name = fileUtils.standard_path_style(
            pathmap.output_dir,
            row,
            sub_directory_addon=sub_directory_addon,
            file_name_addon=file_name_addon)
        #             print(file_name)
        file_names.append(file_name)
    return file_names


def is_result_exist(audio_ins, label_names, v_str, file_properties_df, pathmap):
    file_names = output_file_names(audio_ins, label_names, v_str,
                                   file_properties_df, pathmap)
    for file_name in file_names:
        file_name = file_name.with_suffix('.npy')
        if not file_name.exists():
            return False
    return True


# def main(args):
#     region_location = setup_inputs(args)
#     model_saved, transformCompose, config, file_properties_df, pathmap = setup(
#         args)
#     region_location_datasets = load_audio_files(region_location,
#                                                 file_properties_df)
#     label_names = config['label_names']
#     v_str = config['v_str']
#     for region, location in region_location:
#         print(region, location)
#         region_location_ins = region_location_datasets[(region, location)]
#         region_location_ins.update_samples_w_clipping_info(
#             output_folder=pathmap.clipping_results_path)
#         # print('inference part')
#         for audio_ins in region_location_ins.values():
#             if is_result_exist(audio_ins, label_names, v_str,
#                                file_properties_df, pathmap):
#                 continue
#             dataloader = prepare_dataloader_from_audio_ins(
#                 audio_ins, config, transformCompose)
#             outputs = single_file_inference(dataloader, config, model_saved)
#             save_results_disk(outputs, audio_ins, label_names, v_str,
#                               file_properties_df, pathmap)



In [81]:
class Arguments():
    def __init__(self,index,count,gpu):
        self.index=index
        self.count=count
        self.gpu=gpu

args = Arguments(0,1,1)

In [46]:
# main(args)

In [47]:
# region_location = setup_inputs(args)
model_saved, transformCompose, config, file_properties_df, pathmap = setup(args)



In [48]:
# def load_audio_files(region_location, file_properties_df):
#     '''Load audio files from given regions and locations as Audio dataset
#     '''
#     region_location_datasets = {}
#     for region, location in region_location:
#         filtered_files = file_properties_df[file_properties_df.region == region]
#         filtered_files = filtered_files[filtered_files.locationId == location]
#         filtered_files = filtered_files[filtered_files.durationSec > 0]
#         dataset_name_v = '-'.join([region, location])
#         audio_dataset = dataimport.Dataset(dataset_name_v=dataset_name_v)
#         for i in filtered_files.iterrows():
#             audio_dataset[i[0]] = dataimport.Audio(i[1].name,
#                                                    float(i[1].durationSec))
#         region_location_datasets[(region, location)] = audio_dataset
#     return region_location_datasets

In [49]:
import run

In [50]:
audio_dataset, _ = run.prepare_dataset()


/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite/meganLabeledFiles_wlenV1.txt
/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite
4 files are missing corresponding to excell entries
'-> 5 number of samples are DELETED due to ignore_files and missing_audio_files'
-> 415 samples DELETED because they are not in the excell

-> 0 samples DELETED because they do not have the taxo info coming from excell

-> classes that do not have enough data:
[REMOVED!]

-> classes that have enough data:
['other-biophony'] 56.0
['other-insect'] 140.0
['other-bird'] 661.0
['songbirds'] 392.0
['duck-goose-swan'] 183.0
['grouse-ptarmigan'] 59.0
['other-anthrophony'] 66.0
['other-mammal'] 0.0
['other-silence'] 20.0
['unknown-sound'] 2.0
['other-aircraft'] 107.0
['seabirds'] 1.0
['canids'] 1.0
['loons'] 29.0
['other-car'] 37.0
['other-flare'] 11.0
['other-rain'] 20.0
('-> 0 number of samples are deleted because their taxonomy category does not '
 'have enough data.')
-> classes that do not have e

In [53]:
EXCERPT_LENGTH=10
audio_dataset = run.dataset_generate_samples(audio_dataset,EXCERPT_LENGTH)
all_taxo= list({sound_ins.taxo_code for sound_ins in audio_dataset.values()})

x_data, y_data, location_id_info = run.put_samples_into_array(
    all_taxo, [], audio_dataset)
x_data=np.array(x_data)

In [129]:
names=list(iter(audio_dataset.keys()))

In [159]:
import datetime
timestamps=[]
file_paths = []
audio_index=[]
for sound_ins in audio_dataset.values():
    i=0
    for sample in sound_ins.samples:
        
        day,clock = (sound_ins.name.split('_')[1:3])
        year,month,day = day[:4],day[4:6],day[6:8]
        hour,minute,second = clock[:2],clock[2:4],clock[4:6]
        year,month,day=int(year),int(month),int(day)
        hour,minute,second=int(hour),int(minute),int(second)
        timestamp = datetime.datetime(year,
                                  month,
                                  day,
                                  hour=hour,
                                  minute=minute,
                                  second=second,
                                  microsecond=0)
        timestamp=timestamp+datetime.timedelta(seconds=10*i)
        timestamps.append(timestamp)
        audio_index.append(i)
        file_paths.append(str(sound_ins.path))
        i+=1


'S4A10226_20190709_131602_Bio_Bug18_part-3082.wav'

In [ ]:
#     dataloader = prepare_dataloader_from_audio_ins(
#         audio_ins, config, transformCompose)

In [ ]:
#     outputs = single_file_inference(dataloader, config, model_saved

In [63]:
x_data=np.array(x_data)

In [82]:
dataloader=prepare_dataloader_from_array(x_data[:])

In [83]:
# outputs = single_file_inference(dataloader, config, model_saved)

In [164]:
def sigmoid(data):
    return 1 / (1 + np.exp(-data))

In [84]:
x_data.shape

(3083, 480000)

In [161]:
# def single_file_inference(dataloader, config, model_saved):
outputs = []
activations=[]
for inputs, labels in dataloader['predict']:
#     print(labels)
    del labels
    inputs = inputs.float().to(config['device'])

    output = model_saved(inputs)
    output = output.to('cpu')
    index = output.data.numpy()
    outputs.append(index)
    activations.append(activation['fc1'])
# outputs.append(index)
#     outputs = np.concatenate(outputs)
#     return outputs


In [166]:
aa=sigmoid(outputs[0])

In [192]:
outputs[0][0]

array([  4.7977953,  -1.8959762,  -0.9214242,  -7.415851 ,  -4.75553  ,
        -1.4654305,  -8.198331 , -12.866567 ,  -7.023336 ], dtype=float32)

In [221]:
outputs_list=[]
for out in outputs:
    out_str=[ f"{x:.2f}" for x in list(sigmoid(out[0]))]
    outputs_list.append(out_str)

/scratch/enis/conda/envs/soundenv3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


In [95]:
# model = MyModel()
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

model_saved.fc1.register_forward_hook(get_activation('fc1'))
# x = torch.randn(1, 25)
# output = model(x)
# print(activation['fc2'])

3083

In [91]:
activationsnp=[]
for act in activations:
    act = act.to('cpu')
    act = act.data.numpy()
    activationsnp.append(act)

activationsnp=np.array(activationsnp)

In [113]:
with open('/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite_post/run-5_comic-sweep-29_activations-fc1.npy', 'wb') as f:
    np.save(f,activationsnp)

In [114]:
# len(activationsnp),y_data, location_id_info

In [100]:
loc_per_set = [[
    '12', '14', '27', '49', '31', '39', '44', '45', '48', '19', '16', '21',
    '38', '41', '20', '29', '37', '15'
], ['17', '46', '50', '32', '33', '25', '40'],
               ['11', '18', '34', '24', '13', '22', '36', '47', '30']]

target_taxo = [
    '1.0.0', '1.1.0', '1.1.10', '1.1.7', '0.0.0', '1.3.0', '1.1.8', '0.2.0',
    '3.0.0'
]
multi_label_vector = run.create_multi_label_vector(target_taxo, y_data)

In [107]:
def loc_per_set_find(loc,loc_per_set):
    i2name={0:'train',1:'test',2:'valid'}
    for i,m in enumerate(loc_per_set):
        if loc in m:
            return i2name[i]

In [109]:
set_names=[loc_per_set_find(loc,loc_per_set) for loc in location_id_info]


In [112]:
multi_label_vector[0],set_names[0]

([1, 0, 0, 0, 0, 0, 0, 0, 0], 'valid')

In [215]:
target_taxo_names=[audio_dataset.taxonomy[taxo][0].replace('other-','') for taxo in target_taxo]
target_taxo_names_header=('_pred,'.join(target_taxo_names))+'_pred'
target_taxo_header = (','.join(target_taxo_names))

In [217]:
target_taxo_names_header,target_taxo_header

('biophony_pred,bird_pred,songbirds_pred,duck-goose-swan_pred,anthrophony_pred,insect_pred,grouse-ptarmigan_pred,aircraft_pred,silence_pred',
 'biophony,bird,songbirds,duck-goose-swan,anthrophony,insect,grouse-ptarmigan,aircraft,silence')

In [219]:
# audio_index,file_paths
# with open('/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite_post/run-5_comic-sweep-29_activations-fc1_labels_v2.csv','w') as f:
#     f.writelines('sample_id,file_path,part_index,set_name,location_id,timestamp,'+(','.join(target_taxo))+'\n')
#     for i,label_vector in enumerate(multi_label_vector):
#         label_vector = [str(i) for i in label_vector]
#         t = timestamps[i].strftime('%Y-%m-%d_%H:%M:%S')
#         f.writelines(f'{i},{file_paths[i]},{audio_index[i]},{t},{location_id_info[i]},{set_names[i]},'+(','.join(label_vector))+'\n')

In [226]:
label_vector

['1', '0', '0', '0', '0', '1', '0', '0', '0']

In [227]:
# audio_index,file_paths
with open('/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite_post/run-5_comic-sweep-29_activations-fc1_labels_v3.csv','w') as f:
    f.writelines('sample_id,file_path,part_index,timestamp,location_id,set_name,'+target_taxo_header+','+target_taxo_names_header+'\n')
    for i,label_vector in enumerate(multi_label_vector):
        label_vector = [str(i) for i in label_vector]
        label_vector=','.join(label_vector)
        pred_vector = outputs_list[i]
        
        t = timestamps[i].strftime('%Y-%m-%d_%H:%M:%S')
        f.writelines(f'{i},{file_paths[i]},{audio_index[i]},{t},{location_id_info[i]},{set_names[i]},{(label_vector)},'+(','.join(pred_vector))+'\n')

In [207]:
from nna.exp.megan import preparedataset
preparedataset.load_taxonomy2dataset(('/home/enis/projects/nna/src/nna/assets/taxonomy/taxonomy.yaml'), audio_dataset)


'biophony'

In [115]:
# audio_dataset.update_samples_w_clipping_info(
#     output_folder=pathmap.clipping_results_path)
# # print('inference part')
# for audio_ins in audio_dataset.values():
# #     if is_result_exist(audio_ins, label_names, v_str,
# #                        file_properties_df, pathmap):
# #         continue
#     dataloader = prepare_dataloader_from_audio_ins(
#         audio_ins, config, transformCompose)
#     outputs = single_file_inference(dataloader, config, model_saved)
#     break
    
# #     save_results_disk(outputs, audio_ins, label_names, v_str,
# #                       file_properties_df, pathmap)

In [ ]:
region_location_datasets = load_audio_files(region_location,
                                            file_properties_df)
label_names = config['label_names']
v_str = config['v_str']
# for region, location in region_location:
#     print(region, location)
#     region_location_ins = region_location_datasets[(region, location)]
    region_location_ins.update_samples_w_clipping_info(
        output_folder=pathmap.clipping_results_path)
    # print('inference part')
    for audio_ins in region_location_ins.values():
        if is_result_exist(audio_ins, label_names, v_str,
                           file_properties_df, pathmap):
            continue
        dataloader = prepare_dataloader_from_audio_ins(
            audio_ins, config, transformCompose)
        outputs = single_file_inference(dataloader, config, model_saved)
        save_results_disk(outputs, audio_ins, label_names, v_str,
                          file_properties_df, pathmap)

In [ ]:
# another thing
# with open('/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite/files_filtered_v3_info.csv','w') as f:
#     f.writelines(f'file_name,taxo_code,location_id\n')
#     for file_name in cached_dict.keys():
# #         print(file_name)
#         audio=audio_dataset[file_name.split('/')[-1]]
#         taxo_code=audio.taxo_code
#         location_id=audio.location_id
        
#         f.writelines(f'{file_name},{taxo_code},{location_id}\n')